### 1. Fine tuning модели RuBert на наборе данных RuCola 

In [1]:
import pandas as pd

In [2]:
data_train = pd.read_csv("in_domain_train.csv")
data_train

,id,sentence,acceptable,error_type,detailed_source
0,0,"Вдруг решетка беззвучно поехала в сторону, и н...",1,0,Paducheva2004
1,1,Этим летом не никуда ездили.,0,Syntax,Rusgram
2,2,Только Иван выразил какую бы то ни было готовн...,1,0,Paducheva2013
3,3,"Теперь ты видишь собственными глазами, как тут...",1,0,Paducheva2010
4,4,На поверку вся теория оказалась полной чепухой.,1,0,Paducheva2010
...,...,...,...,...,...
7864,7864,Установки не было введено в действие.,0,Semantics,Paducheva2004
7865,7865,"Конечно, против такой системы ценностей решите...",0,Semantics,Paducheva2013
7866,7866,Симптомов болезни не исчезло.,0,Semantics,Paducheva2013
7867,7867,Послезавтра температура у больного снижается д...,0,Semantics,Rusgram


In [3]:
data_dev = pd.read_csv("in_domain_dev.csv")
data_dev

,id,sentence,acceptable,error_type,detailed_source
0,0,Иван вчера не позвонил.,1,0,Paducheva2013
1,1,"У многих туристов, кто посещают Кемер весной, ...",0,Syntax,USE8
2,2,Лесные запахи набегали волнами; в них смешалос...,1,0,USE5
3,3,Вчера президент имел неофициальную беседу с ан...,1,0,Seliverstova
4,4,Коллега так и не признал вину за катастрофу пе...,1,0,Testelets
...,...,...,...,...,...
978,978,Мысли отказываются остановиться на всяком пред...,0,Semantics,Paducheva2013
979,979,"Не должно быть подозрений, что судью привлекаю...",0,Semantics,Paducheva2013
980,980,"Хорошо, что он купил что-нибудь.",0,Semantics,Rusgram
981,981,"Если бы я не потерял очков, не пришлось бы пок...",0,Semantics,Paducheva2013


In [4]:
from datasets import Dataset

c:\Users\MVKiselev\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
dataset = Dataset.from_pandas(data_train).train_test_split(test_size=0.2, seed=1)
dataset = dataset.rename_column('acceptable', 'label')
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence', 'label', 'error_type', 'detailed_source'],
        num_rows: 6295
    })
    test: Dataset({
        features: ['id', 'sentence', 'label', 'error_type', 'detailed_source'],
        num_rows: 1574
    })
})

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertForSequenceClassification

In [7]:
base_model = 'ai-forever/ruBert-base'

In [8]:
tokenizer = AutoTokenizer.from_pretrained(base_model)

c:\Users\MVKiselev\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
data_tokenized = dataset.map(lambda x: tokenizer(x['sentence'], truncation=True, max_length=512), batched=True, remove_columns=['id', 'sentence', 'error_type', 'detailed_source'])
data_tokenized

Map: 100%|██████████| 1574/1574 [00:00<00:00, 15739.11 examples/s]


DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6295
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1574
    })
})

In [11]:
from transformers import DataCollatorWithPadding

In [12]:
collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
from torch.utils.data import DataLoader

In [14]:
train_dataloader = DataLoader(data_tokenized['train'], shuffle=True, batch_size=4, collate_fn=collator)
val_dataloader = DataLoader(data_tokenized['test'], shuffle=False, batch_size=4, collate_fn=collator)

In [15]:
#model = BertForSequenceClassification.from_pretrained(base_model, num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained(base_model, num_labels=2)
model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [16]:
import torch
import pandas as pd
import numpy as np
from tqdm.auto import tqdm, trange
from torch.optim import Adam

In [17]:
optimizer = Adam(model.parameters(), lr=1e-6)

In [18]:
if torch.cuda.is_available():
    model.cuda()

In [19]:
losses = []
for epoch in trange(3):
    pbar = tqdm(train_dataloader)
    model.train()
    for i, batch in enumerate(pbar):
        out = model(**batch.to(model.device))
        out.loss.backward()
        if i % 1 == 0:
            optimizer.step()
            optimizer.zero_grad()
        losses.append(out.loss.item())
        pbar.set_description(f'loss: {np.mean(losses[-100:]):2.2f}')
    model.eval()
    eval_losses = []
    eval_preds = []
    eval_targets = []
    for batch in tqdm(val_dataloader):
        with torch.no_grad():
                out = model(**batch.to(model.device))
        eval_losses.append(out.loss.item())
        eval_preds.extend(out.logits.argmax(1).tolist())
        eval_targets.extend(batch['labels'].tolist())
    print('recent train loss', np.mean(losses[-100:]), 'eval loss', np.mean(eval_losses), 'accuracy', np.mean(np.array(eval_targets) == eval_preds))

  0%|          | 0/3 [00:00<?, ?it/s]c:\Users\MVKiselev\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
 33%|███▎      | 1/3 [07:54<15:49, 474.60s/it]

recent train loss 0.5432492765784264 eval loss 0.5517015452993098 accuracy 0.7465057179161372


 67%|██████▋   | 2/3 [16:59<08:36, 516.03s/it]

recent train loss 0.5714918261766434 eval loss 0.5377188452004176 accuracy 0.7592121982210928


100%|██████████| 3/3 [26:24<00:00, 528.18s/it]

recent train loss 0.4805538737773895 eval loss 0.5404081200131305 accuracy 0.7649301143583227


In [20]:
dataset_dev = Dataset.from_pandas(data_dev)
dataset_dev = dataset_dev.rename_column('acceptable', 'label')
dataset_dev

Dataset({
    features: ['id', 'sentence', 'label', 'error_type', 'detailed_source'],
    num_rows: 983
})

In [22]:
test_data_tokenized = dataset_dev.map(lambda x: tokenizer(x['sentence'], truncation=True, max_length=512), batched=True, remove_columns=['id', 'sentence', 'error_type', 'detailed_source'])
test_data_tokenized

Map: 100%|██████████| 983/983 [00:00<00:00, 23986.14 examples/s]


Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 983
})

In [24]:
test_dataloader = DataLoader(test_data_tokenized, shuffle=True, batch_size=4, collate_fn=collator)

In [25]:
model.eval()
eval_losses = []
eval_preds = []
eval_targets = []
for batch in tqdm(test_dataloader):
    with torch.no_grad():
            out = model(**batch.to(model.device))
    eval_losses.append(out.loss.item())
    eval_preds.extend(out.logits.argmax(1).tolist())
    eval_targets.extend(batch['labels'].tolist())
print('recent train loss', np.mean(losses[-100:]), 'eval loss', np.mean(eval_losses), 'accuracy', np.mean(np.array(eval_targets) == eval_preds))

100%|██████████| 246/246 [00:07<00:00, 33.96it/s]

recent train loss 0.4805538737773895 eval loss 0.5220698794213737 accuracy 0.7639877924720244


### 2. RuGPT3

[Базовая статья](https://cloud.ru/ru/datahub/rugpt3family/rugpt-3-large)

[Пример кода](https://colab.research.google.com/github/sberbank-ai/ru-gpts/blob/master/examples/Generate_text_with_RuGPTs_HF.ipynb)

In [53]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name_or_path = "ai-forever/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).cuda()

Проверка

In [54]:

text = "Александр Сергеевич Пушкин родился в "
input_ids = tokenizer.encode(text, return_tensors="pt").cuda()
out = model.generate(input_ids.cuda())
generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)

c:\Users\MVKiselev\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Александр Сергеевич Пушкин родился в 
1799 году в Москве.

В 1820 году,


Промптинг

In [55]:
text = "Текст: Шла собака по роялю.\n Классификация: допустимый\n Текст: Машинное обучение это круто\n Классификация:"
input_ids = tokenizer.encode(text, return_tensors="pt").cuda()
out = model.generate(input_ids.cuda(), max_new_tokens=200, )
generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)

Текст: Шла собака по роялю.
 Классификация: допустимый
 Текст: Машинное обучение это круто
 Классификация: допустимый
 Текст: В чем отличие между человеком и компьютером
 Текст: Как сделать так, чтобы в браузере не было рекламы
 Текст: Как сделать так, чтобы в браузере не было рекламы
 Текст: Как сделать так, чтобы в браузере не было рекламы
 Текст: Как сделать так, чтобы в браузере не было рекламы
 Текст: Как сделать так, чтобы в браузере не было рекламы
 Текст: Как сделать так, чтобы в браузере не было рекламы
 Текст: Как сделать так, чтобы в браузере не было рекламы
 Текст: Как сделать так, чтобы в браузере не было рекламы
 Текст: Как сделать так, чтобы в браузере не было рекламы
 Текст: Как сделать так, чтобы в браузере не было рекламы
 Текст: Как сделать так, чтобы в браузере не было рекламы
 Текст: Как сделать так, чтобы в браузере не было рекламы
 Текст: Как сделать так, чтобы в браузере не было рекламы
 Текст: Как сделать так


In [56]:
text = "Текст: Шла собака по роялю.\n Классификация: допустимый\n Текст: Собака сено лежала\n Классификация:"
input_ids = tokenizer.encode(text, return_tensors="pt").cuda()
out = model.generate(input_ids.cuda(), max_new_tokens=200, )
generated_text = list(map(tokenizer.decode, out))[0][0:135]
print(generated_text)

Текст: Шла собака по роялю.
 Классификация: допустимый
 Текст: Собака сено лежала
 Классификация: допустимый
 Текст: Собака сено лежала


In [57]:
text = "Текст: Шла собака по роялю.\n Классификация: допустимый\n Текст: Собака сено лежала\n Классификация: недопустимый\n Текст: Собаки стая травили волки\n Классификация:"
input_ids = tokenizer.encode(text, return_tensors="pt").cuda()
out = model.generate(input_ids.cuda(), max_new_tokens=200, )
generated_text = list(map(tokenizer.decode, out))[0][0:180]
print(generated_text)

Текст: Шла собака по роялю.
 Классификация: допустимый
 Текст: Собака сено лежала
 Классификация: недопустимый
 Текст: Собаки стая травили волки
 Классификация: недопустимый
 Текст


Вывод: Из приведённого промптинга видно, что модель обучаема: обучение на фразе "Собака сено лежала" позволило правильно классифицировать фразу "Собаки стая травили волки"
Модель вполне работоспособна и пригодна для дальнейшего дообучения